In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pymap3d as pm

In [2]:
df = pd.read_csv("/home/torsten/ms/results/fg_gnss.csv", header=None, skiprows=[])
df

,0,1,2,3,4
0,1,0,0.000316,-0.000831,0.000140
1,1,1,4.622940,-0.007665,-0.216009
2,2,0,0.000746,-0.001908,0.000313
3,2,1,4.517080,-0.022877,-0.065522
4,2,2,9.225580,0.031795,-0.319978
...,...,...,...,...,...
4554,94,90,173.946000,-59.679000,-12.481700
4555,94,91,173.909000,-59.577900,-12.514800
4556,94,92,173.870000,-59.498900,-12.543300
4557,94,93,173.858000,-59.444500,-12.569800


In [3]:
# -33.7888315,18.9602664,156.662
lat0 = -33.7888315
lon0 = 18.9602664
h0 = 156.662

df_gnss = pd.read_csv("/home/torsten/ms/data/rosbag2_2024_01_05-12_47_24/ublox.csv")
df_gnss

,sec,nanosec,lat,lon,alt,vel_n,vel_e,vel_d,h_acc
0,1704451644,745270590,-33.788448,18.960100,149.408,0.00,0.03,0.04,15.685
1,1704451645,150504223,-33.788447,18.960102,149.261,0.03,0.02,0.04,15.708
2,1704451645,545843986,-33.788448,18.960107,148.920,-0.08,0.11,0.05,15.730
3,1704451645,946381040,-33.788448,18.960107,148.806,-0.03,0.04,0.05,15.754
4,1704451646,349933946,-33.788448,18.960109,148.638,-0.07,0.06,0.05,15.775
...,...,...,...,...,...,...,...,...,...
1564,1704452276,352632009,-33.789307,18.962076,146.269,0.01,0.01,0.03,4.062
1565,1704452276,749299056,-33.789307,18.962076,146.309,0.03,0.03,0.03,4.061
1566,1704452277,154872502,-33.789307,18.962075,146.354,0.01,0.01,0.03,4.058
1567,1704452277,552798830,-33.789307,18.962075,146.424,0.01,0.00,0.03,4.055


In [4]:
# Sat image segments
segment_filenames = ["1.csv", "2.csv", "3.csv", "4.csv"]
segment_coords = []
segments_enu = []

for fn in segment_filenames:
    df_seg = pd.read_csv("~/ms/data/old_zoo_sat_paths/" + fn)
    segment_coords.append(df_seg.iloc[:, 0:2].values)

for seg in segment_coords:
    segments_enu.append(np.zeros(seg.shape))
    for i in range(seg.shape[0]):
        enu = pm.geodetic2enu(seg[i, 1], seg[i, 0], 102, lat0, lon0, h0)
        segments_enu[-1][i, 0] = enu[0]
        segments_enu[-1][i, 1] = enu[1]
    

In [5]:
%matplotlib tk
start = 0

plt.figure()
plt.axis('equal')
plt.grid()

num_sets = df.iloc[-1,0]

# Factor graph
for i in range(num_sets):
    plt.plot(df.iloc[start:start+i+2, 2], df.iloc[start:start+i+2, 3], "-x", color="grey")
    plt.text(df.iloc[start+i+1, 2], df.iloc[start+i+1, 3], str(i+1))
    
    start+=i+2

# GNSS trail
for index, row in df_gnss.iterrows():
    if index % 10 != 0:
        continue
    enu = pm.geodetic2enu(row["lat"], row["lon"], row["alt"], lat0, lon0, h0)
    plt.plot(enu[0], enu[1], "-bx")
    plt.errorbar(enu[0], enu[1], row["h_acc"], row["h_acc"])

# Sat image segments
for seg in segments_enu:
    #plt.plot(seg[:, 0], seg[:, 1], "-", color="blue")
    pass

plt.show()